<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

# Data Formatting (categorical)


In this formatting tutorial we will see the categorical case.


Let's get [some data](https://en.wikipedia.org/wiki/List_of_freedom_indices):

In [1]:
import pandas as pd

link='https://en.wikipedia.org/wiki/List_of_freedom_indices'
freeDFs=pd.read_html(link,flavor='bs4',match='w',attrs={'class':"wikitable"})

# how many tables?
len(freeDFs)

ImportError: html5lib not found, please install it

In [ ]:
#is this one?
freeDFs[0]

Then, you want the second table:

In [ ]:
allFree=freeDFs[1]
allFree.head()

Cleaning column names:

In [ ]:
allFree.columns

This is a good alternative:

In [ ]:
allFree.columns.str.replace(r"\W|\d","",regex=True)

In [ ]:
#then
allFree.columns=allFree.columns.str.replace(r"\W|\d","",regex=True)

Let's clean all the leading/trailing space in every cell:

In [ ]:
# this code breaks if applied to numeric columns
allFree=allFree.apply(lambda x: x.str.strip())

Do we have unique country names?

In [ ]:
len(allFree.Country)==len(pd.unique(allFree.Country))

Let's start formatting:

In [ ]:
# check current data types:
allFree.info()

We see every column is just of the object type. Instead of frequency tables, check unique values:

In [ ]:
[list(allFree[c].sort_values().unique()) for c in allFree.columns[1:]]

You wanted to check for mistakes like ['free', 'not free', 'partly free', 'Free']. The cells are clean.

Now, let's turn the values into **ordinal** categories. Remember that the worst, best and middle values should be comparable:

In [ ]:
mapper1 = {'not free': 1,'partly free': 3, 'free':5 }
allFree.FreedomintheWorld.replace(mapper1,inplace=True)

mapper2 = {'repressed':1, 'mostly unfree':2,'moderately free':3, 'mostly free':4, 'free':5}
allFree.IndexofEconomicFreedom.replace(mapper2,inplace=True)


mapper3 = {'very serious situation':1,'difficult situation':2, 'noticeable problems':3,
           'satisfactory situation':4,'good situation':5}
allFree.PressFreedomIndex.replace(mapper3,inplace=True)

mapper4 = {'authoritarian regime':1, 'hybrid regime':2,'flawed democracy':3,'full democracy':5}
allFree.DemocracyIndex.replace(mapper4,inplace=True)


In [ ]:
allFree

Let's explore:

In [ ]:
#check types:
allFree.info()

In [ ]:
# what about
allFree[allFree.columns[1:]]=allFree.iloc[:,1:].apply(lambda x: x.astype('Int64'))

In [ ]:
#then
allFree.info()

However, these are not yet ordinal. Let's do it:

In [ ]:
from pandas.api.types import CategoricalDtype

order = CategoricalDtype(categories=[1,2,3,4,5], ordered=True)
allFree.iloc[:,1:].apply(lambda x:x.astype(order)).info()

In [ ]:
# create some new names:
newNames=allFree.columns[1:]+'_ord'
#see
newNames

In [ ]:
allFree[newNames]=allFree.iloc[:,1:].apply(lambda x:x.astype(order))

In [ ]:
allFree.info()

In [ ]:
allFree.IndexofEconomicFreedom_ord

You may want to rename them:

In [ ]:
ordCats={1:'veryLow',2:'low',3:'medium',4:'good',5:'veryGood'}

turnToOrdinal= lambda x:x.cat.rename_categories(ordCats)

allFree.iloc[:,5:].apply(turnToOrdinal)

In [ ]:
allFree[newNames]=allFree.iloc[:,5:].apply(turnToOrdinal)

# see
allFree.head(10)

Let's keep this last result, but this let me show you the use of **pickle** format:

In [ ]:
#saving

import os 

allFree.to_csv(os.path.join("data","allFree.csv"),index=False )
allFree.to_pickle(os.path.join("data","allFree.pkl") )

In [ ]:
#reading

dfPickle=pd.read_pickle(os.path.join("data","allFree.pkl") )  
dfCSV=pd.read_csv(os.path.join("data","allFree.csv") )  

Now, notice the difference when you have categorical data:

In [ ]:
dfPickle.info()

In [ ]:
dfCSV.info()

In [ ]:
# the file kept the data type
dfPickle.DemocracyIndex_ord

In [ ]:
# the file did not keep the data type
dfCSV.DemocracyIndex_ord